# Define Parcel APNs and boundaries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import os

tqdm.pandas()

In [2]:
# Paths
OAK_FP = '/oak/stanford/groups/deho/building_compliance/'
OUTPUT_FP = os.path.join(OAK_FP, 'outputs', 'Population-Estimates', 'outputs')
INPUT_FP = os.path.join(OAK_FP, 'outputs', 'Population-Estimates', 'inputs')

In [3]:
# Raw permit data (subset to AANC)
bldg_recent = gpd.read_file(os.path.join(OAK_FP, 'permits', 'bldg_recent.geojson'))
bldg_active = gpd.read_file(os.path.join(OAK_FP, 'permits', 'bldg_active.geojson'))
bldg_expired = gpd.read_file(os.path.join(OAK_FP, 'permits', 'bldg_expired.geojson'))

bldg_active['OBJECTID'] = bldg_active['OBJECTID'].astype(str).apply(lambda x: f'A_{x}')
bldg_recent['OBJECTID'] = bldg_recent['OBJECTID'].astype(str).apply(lambda x: f'R_{x}')
bldg_expired['OBJECTID'] = bldg_expired['OBJECTID'].astype(str).apply(lambda x: f'E_{x}')

cols = list(bldg_active.columns) + ['permit_status']
permits_raw = gpd.GeoDataFrame(columns=cols)
for i, status in zip([bldg_active, bldg_recent, bldg_expired], ['active', 'recent', 'expired']):
    i['permit_status'] = status
    # Subset
    i = i[i['WORKDESC'].isin(['New Construction', 'Additions/Alterations'])]
    permits_raw = pd.concat([permits_raw, i])
permits_raw = permits_raw.reset_index(drop=True)

permits_raw.crs = "EPSG:4326"
permits_raw['year'] = permits_raw['FOLDERNUM'].apply(lambda x: int(x.split('-')[0]))

In [4]:
# Our current data (just AA+NC permits)
permits = pd.read_csv(os.path.join(OAK_FP, 'outputs/Permit-Matching/outputs/all_permits.csv'))
permits['geometry_permit'] = gpd.GeoSeries.from_wkt(permits['geometry_permit'])
permits.drop_duplicates(inplace=True)

permits_raw.crs = "EPSG:4326"

In [5]:
permits_raw.head(1)

,OBJECTID,FACILITYID,INTID,ADDRESS,APN,APPLICANT,CONTRACTOR,WORKDESC,SUBDESC,PERMITAPPROVAL,...,PERMITVALUE,SQUAREFOOT,FOLDERNUM,LASTUPDATE,LASTEDITOR,NOTES,ENTERPRISEID,geometry,permit_status,year
0,A_294,351,351,"2027 LOCKWOOD DR , SAN JOSE CA 95132-1323",09226077,CHRISTOPHER CAO,None,New Construction,2nd Unit Added,"B-Complete, E-Complete, M-Complete, P-Complete",...,71070.0,600,2018-109724-RS,1.622107e+12,FME,,PLN-PBAC-0000000351,POINT (-121.86056 37.41525),active,2018


In [6]:
# SJ Residential parcels
sj_parcels_cbgs = gpd.read_file(os.path.join(OUTPUT_FP, 'sj-parcels-res-cbgs'))

In [8]:
length = sj_parcels_cbgs.to_crs('EPSG:26910')['geometry'].length

In [9]:
length.describe()

count    159930.000000
mean        106.110200
std          27.417965
min           6.374590
25%          97.517640
50%         101.359958
75%         111.801065
max        1923.333389
dtype: float64

In [11]:
area = sj_parcels_cbgs.to_crs('EPSG:26910')['geometry'].area

In [12]:
area.describe()

count    159930.000000
mean        649.345023
std         310.211776
min           1.102598
25%         550.736357
50%         589.161898
75%         703.936382
max        4991.710854
dtype: float64

In [5]:
# Assessor parcel data
assessor_parcels = gpd.read_file(os.path.join(INPUT_FP, 'AssessorData', 'assessor_scc_parcels.geojson'))
sj_assessor_parcels = assessor_parcels.loc[assessor_parcels['SITUS_CITY'] == 'SAN JOSE'].copy()

In [ ]:
sj_assessor_parcels.head()

In [154]:
# Keep only parcel data
sj_assessor_parcels = sj_assessor_parcels.loc[sj_assessor_parcels['FEATURE'] == 'PAR']

sj_assessor_parcels = sj_assessor_parcels[[
    'OBJECTID', 'APN', 'SITUS_HOUS', 'SITUS_STRE', 'SITUS_HO_1', 'SITUS_ST_1', 'SITUS_ST_2', 'SITUS_UNIT', 
    'SITUS_CITY', 'SITUS_STAT', 'SITUS_ZIP_', 'geometry'
]].copy()

In [155]:
# Configure address
sj_assessor_parcels['ADDRESS'] = sj_assessor_parcels.apply(
    lambda row: '{} {} {} {} {} , {} {} {}'.format(
        str(row['SITUS_HOUS'] or ''), 
        str(row['SITUS_STRE'] or ''), 
        #str(row['SITUS_HO_1'] or ''), 
        str(row['SITUS_ST_1'] or ''), 
        str(row['SITUS_ST_2'] or ''), 
        str(row['SITUS_UNIT'] or ''), 
        str(row['SITUS_CITY'] or ''), 
        str(row['SITUS_STAT'] or ''), 
        str(row['SITUS_ZIP_'] or '')), 
    axis=1)
sj_assessor_parcels.rename(columns={'APN': 'ASSESSOR APN'}, inplace=True)

In [156]:
# Handle address duplicates matching to different APNs
# For now, drop all parcels that have duplicated addresses. 
print('[INFO] Duplicated addresses: {}'.format(sj_assessor_parcels.duplicated('ADDRESS').sum()))
sj_assessor_parcels = sj_assessor_parcels.drop_duplicates(subset=['ADDRESS'], keep=False)

[INFO] Duplicated addresses: 2589


In [157]:
# Check how many spatially-matched APNs match to assessor APNs
# These are the 14,372 AANC permits in San Jose during the entire period. 
merged_permits = pd.merge(
    permits_raw[['OBJECTID', 'APN', 'WORKDESC', 'ADDRESS', 'geometry']], 
    permits[['OBJECTID_left', 'APN_parcel', 'geometry_parcel', 'issue_year']], 
    how='left', left_on='OBJECTID', right_on='OBJECTID_left', validate='one_to_one'
)
(merged_permits['APN'] != merged_permits['APN_parcel']).sum()
merged_permits.rename(columns={'APN': 'Permit APN', 'APN_parcel': 'SM APN'}, inplace=True)

In [143]:
#dupli = sj_assessor_parcels[sj_assessor_parcels.duplicated(subset=['ADDRESS'])]

In [158]:
# Obtain Assessor APN based on permit address
# First merge
merged_permits = pd.merge(
    merged_permits, sj_assessor_parcels[['ASSESSOR APN', 'ADDRESS']], how='left', 
    on='ADDRESS', validate='many_to_one')

In [159]:
merged_permits.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 14372 entries, 0 to 14371
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   OBJECTID         14372 non-null  object  
 1   Permit APN       14175 non-null  object  
 2   WORKDESC         14372 non-null  object  
 3   ADDRESS          14372 non-null  object  
 4   geometry         14372 non-null  geometry
 5   OBJECTID_left    14372 non-null  object  
 6   SM APN           14232 non-null  object  
 7   geometry_parcel  14261 non-null  object  
 8   issue_year       14372 non-null  int64   
 9   ASSESSOR APN     9941 non-null   object  
dtypes: geometry(1), int64(1), object(8)
memory usage: 1.2+ MB


In [160]:
# We perform a second merge replacing full zip code with 5-digit zip code, as this
# is missing from some entries of the permit data and not others. 
sj_assessor_parcels_av2 = sj_assessor_parcels.copy()
sj_assessor_parcels_av2['ADDRESS'] = sj_assessor_parcels_av2.apply(
    lambda row: '{} {} {} {} {} , {} {} {}'.format(
        str(row['SITUS_HOUS'] or ''), 
        str(row['SITUS_STRE'] or ''), 
        #str(row['SITUS_HO_1'] or ''), 
        str(row['SITUS_ST_1'] or ''), 
        str(row['SITUS_ST_2'] or ''), 
        str(row['SITUS_UNIT'] or ''), 
        str(row['SITUS_CITY'] or ''), 
        str(row['SITUS_STAT'] or ''), 
        str(row['SITUS_ZIP_'].split('-')[0] or '')), 
    axis=1)
sj_assessor_parcels_av2.rename(columns={'ASSESSOR APN': 'ASSESSOR APN2'}, inplace=True)
sj_assessor_parcels_av2.drop_duplicates(subset=['ADDRESS'], keep=False, inplace=True)

merged_permits = pd.merge(
    merged_permits, sj_assessor_parcels_av2[['ASSESSOR APN2', 'ADDRESS']], how='left', 
    on='ADDRESS', validate='many_to_one')

In [202]:
merged_permits.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 14372 entries, 0 to 14371
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   OBJECTID         14372 non-null  object  
 1   Permit APN       14175 non-null  object  
 2   WORKDESC         14372 non-null  object  
 3   ADDRESS          14372 non-null  object  
 4   geometry         14372 non-null  geometry
 5   OBJECTID_left    14372 non-null  object  
 6   SM APN           14232 non-null  object  
 7   geometry_parcel  14261 non-null  object  
 8   issue_year       14372 non-null  int64   
 9   ASSESSOR APN     9966 non-null   object  
 10  ASSESSOR APN3    9966 non-null   object  
dtypes: geometry(1), int64(1), object(9)
memory usage: 1.3+ MB


In [201]:
# Combine Assessor APNS
merged_permits['ASSESSOR APN'] = merged_permits.apply(
    lambda row: row['ASSESSOR APN2'] if pd.isnull(row['ASSESSOR APN']) else row['ASSESSOR APN'], axis=1)
merged_permits.drop('ASSESSOR APN2', inplace=True, axis=1)

In [212]:
# Impute the remaining Assessor APNs spacially
spatial_assesssor_permits = permits_raw.copy()
spatial_assesssor_permits = spatial_assesssor_permits.sjoin(
    sj_assessor_parcels[['ASSESSOR APN', 'ADDRESS', 'geometry']], 
    predicate='intersects', how='left')

spatial_assesssor_permits = spatial_assesssor_permits.loc[
    ~spatial_assesssor_permits['ASSESSOR APN'].isna()]

spatial_assesssor_permits.rename(
    columns={'ASSESSOR APN': 'SM ASSESSOR APN', 'ADDRESS_right': 'SM ASSESSOR ADRRESS'}, inplace=True)

In [213]:
merged_permits = pd.merge(
    merged_permits, spatial_assesssor_permits[['OBJECTID', 'SM ASSESSOR APN', 'SM ASSESSOR ADRRESS']], 
    how='left', on='OBJECTID', validate='one_to_one')

In [216]:
merged_permits.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 14372 entries, 0 to 14371
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   OBJECTID             14372 non-null  object  
 1   Permit APN           14175 non-null  object  
 2   WORKDESC             14372 non-null  object  
 3   ADDRESS              14372 non-null  object  
 4   geometry             14372 non-null  geometry
 5   OBJECTID_left        14372 non-null  object  
 6   SM APN               14232 non-null  object  
 7   geometry_parcel      14261 non-null  object  
 8   issue_year           14372 non-null  int64   
 9   ASSESSOR APN         9966 non-null   object  
 10  SM ASSESSOR APN      13170 non-null  object  
 11  SM ASSESSOR ADRRESS  13170 non-null  object  
dtypes: geometry(1), int64(1), object(10)
memory usage: 1.4+ MB


In [217]:
(merged_permits['ASSESSOR APN'] == merged_permits['SM ASSESSOR APN']).sum()

9908

In [226]:
# Summary stats
summary_stats = merged_permits.copy()

# Subset to permits issued 2015-2020
summary_stats = summary_stats.loc[
    summary_stats['issue_year'].isin([2015, 2016, 2017, 2018, 2019, 2020])]

# Subset to residential parcels
summary_stats = summary_stats.loc[summary_stats['SM APN'].isin(sj_parcels_cbgs['APN'])]

In [240]:
summary_stats['WORKDESC'].unique()

array(['New Construction', 'Additions/Alterations'], dtype=object)

In [227]:
summary_stats.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3176 entries, 0 to 14369
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   OBJECTID             3176 non-null   object  
 1   Permit APN           3171 non-null   object  
 2   WORKDESC             3176 non-null   object  
 3   ADDRESS              3176 non-null   object  
 4   geometry             3176 non-null   geometry
 5   OBJECTID_left        3176 non-null   object  
 6   SM APN               3176 non-null   object  
 7   geometry_parcel      3176 non-null   object  
 8   issue_year           3176 non-null   int64   
 9   ASSESSOR APN         2872 non-null   object  
 10  SM ASSESSOR APN      3141 non-null   object  
 11  SM ASSESSOR ADRRESS  3141 non-null   object  
dtypes: geometry(1), int64(1), object(10)
memory usage: 322.6+ KB


In [234]:
(summary_stats['SM APN'] == summary_stats['SM ASSESSOR APN']).sum()

3129

In [235]:
(summary_stats['SM APN'] == summary_stats['Permit APN']).sum()

2967

In [237]:
(summary_stats['SM APN'] == summary_stats['ASSESSOR APN']).sum()

2861

Prepare Batch #2 with those that do match

In [238]:
batch2 = summary_stats.copy()
batch2 = batch2.loc[(batch2['SM APN'] == batch2['SM ASSESSOR APN'])]

In [239]:
batch2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3129 entries, 0 to 14369
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   OBJECTID             3129 non-null   object  
 1   Permit APN           3124 non-null   object  
 2   WORKDESC             3129 non-null   object  
 3   ADDRESS              3129 non-null   object  
 4   geometry             3129 non-null   geometry
 5   OBJECTID_left        3129 non-null   object  
 6   SM APN               3129 non-null   object  
 7   geometry_parcel      3129 non-null   object  
 8   issue_year           3129 non-null   int64   
 9   ASSESSOR APN         2863 non-null   object  
 10  SM ASSESSOR APN      3129 non-null   object  
 11  SM ASSESSOR ADRRESS  3129 non-null   object  
dtypes: geometry(1), int64(1), object(10)
memory usage: 317.8+ KB


In [249]:
idx = np.random.choice(range(len(batch2)), size=5, replace=False)
batch2.iloc[idx]['SM APN']

11184    24408036
12789    46239021
9309     27451032
9935     49722035
9477     26452036
Name: SM APN, dtype: object

In [243]:
summary_stats.loc[(summary_stats['SM APN'] != summary_stats['SM ASSESSOR APN']) & (~summary_stats['SM ASSESSOR APN'].isna())]

,OBJECTID,Permit APN,WORKDESC,ADDRESS,geometry,OBJECTID_left,SM APN,geometry_parcel,issue_year,ASSESSOR APN,SM ASSESSOR APN,SM ASSESSOR ADRRESS
97,A_17385,47217024,New Construction,"737 S 2ND ST 1 , SAN JOSE CA 95112-5821",POINT (-121.88142 37.32549),A_17385,47217097,POLYGON ((-121.88142073312616 37.3254265513074...,2019,NaN,47217024,"735 S 2ND ST , SAN JOSE CA 95112-5821"
252,A_27551,24123053,New Construction,"1598 RINGWOOD AV , SAN JOSE CA 95131",POINT (-121.88998 37.38470),A_27551,24123055,POLYGON ((-121.88984001941922 37.3847110639443...,2020,NaN,24123054,"1508 MURPHY AV , SAN JOSE CA 95131-2425"
9586,E_27943,47765103,Additions/Alterations,"1370 MCQUESTEN DR A , SAN JOSE CA 95122-4103",POINT (-121.84625 37.32973),E_27943,47765C02,POLYGON ((-121.84642498726052 37.3300087227158...,2015,NaN,47765103,"1370 MCQUESTEN DR A , SAN JOSE CA 95122"
9735,E_28994,45825054,Additions/Alterations,"4607 BLACK RIVER CT , SAN JOSE CA 95136-2703",POINT (-121.86338 37.26492),E_28994,45825C02,POLYGON ((-121.86334267247558 37.2642760592608...,2015,45825054,45825054,"4607 BLACK RIVER CT , SAN JOSE CA 95136-2703"
10521,E_35059,42907004,Additions/Alterations,"1127 MEREDITH AV , SAN JOSE CA 95125-3239",POINT (-121.90108 37.30646),E_35059,42907004,POLYGON ((-121.90086127756395 37.3063892667020...,2017,42907004,42907005,"1131 MEREDITH AV , SAN JOSE CA 95125-3239"
11489,E_41806,59229011,Additions/Alterations,"341 PASQUALE CT , SAN JOSE CA 95133-2435",POINT (-121.84070 37.37271),E_41806,59229C01,POLYGON ((-121.84054863641344 37.3729762008285...,2018,59229011,59229011,"341 PASQUALE CT , SAN JOSE CA 95133-2435"
12199,E_46085,58132015,Additions/Alterations,"1327 HILLCREST DR , SAN JOSE CA 95120-5618",POINT (-121.87184 37.20922),E_46085,58132033,POLYGON ((-121.87154132785456 37.2093199696808...,2018,58132015,58132015,"1327 HILLCREST DR , SAN JOSE CA 95120-5618"
12752,E_53362,42916069,Additions/Alterations,"1137 BRACE AV 1 , SAN JOSE CA 95125-3249",POINT (-121.90042 37.30430),E_53362,42916C04,POLYGON ((-121.9006446012302 37.30476477418398...,2019,42916069,42916069,"1137 BRACE AV 1 , SAN JOSE CA 95125-3249"
13022,E_54832,46755001,Additions/Alterations,"388 S 11TH ST , SAN JOSE CA 95112-2258",POINT (-121.87524 37.33526),E_54832,46755C01,POLYGON ((-121.87513066307586 37.3355753662554...,2019,46755001,46755001,"388 S 11TH ST , SAN JOSE CA 95112-2258"
13143,E_55569,70449008,Additions/Alterations,"6145 CAMINO VERDE DR A , SAN JOSE CA 95119-1400",POINT (-121.80184 37.23535),E_55569,70449C01,POLYGON ((-121.80265856742722 37.2355483179328...,2019,70449008,70449008,"6145 CAMINO VERDE DR A , SAN JOSE CA 95119-1400"


In [178]:
merged_permits.loc[merged_permits['ASSESSOR APN'].isna()].iloc[5:10]

,OBJECTID,Permit APN,WORKDESC,ADDRESS,geometry,OBJECTID_left,SM APN,geometry_parcel,issue_year,ASSESSOR APN,ASSESSOR APN2,ASSESSOR APN3
37,A_5501,43955059,Additions/Alterations,"1375 DRY CREEK RD , SAN JOSE CA 95125-0000",POINT (-121.90136 37.29156),A_5501,43955059,POLYGON ((-121.90130336802491 37.2917911429285...,2017,NaN,NaN,NaN
42,A_7328,24963032,New Construction,"450 N 21ST ST , SAN JOSE CA 95112-1747",POINT (-121.87626 37.35325),A_7328,NaN,NaN,2020,NaN,NaN,NaN
43,A_7329,43955021,New Construction,"1930 HICKS AV 2 , SAN JOSE CA 95125",POINT (-121.90217 37.29014),A_7329,43955021,POLYGON ((-121.9018919311962 37.29033066719882...,2020,NaN,NaN,NaN
44,A_7672,48126030,New Construction,"245 MCCREERY AV 2 , SAN JOSE CA 95116",POINT (-121.84820 37.35035),A_7672,48126030,POLYGON ((-121.84817900902405 37.3504477973360...,2020,NaN,NaN,NaN
48,A_8396,49738004,New Construction,"2660 MONTEREY RD , SAN JOSE CA 95111-3050",POINT (-121.85305 37.29673),A_8396,49738025,POLYGON ((-121.85368860061926 37.2963199807144...,2018,NaN,NaN,NaN


In [193]:
merged_permits.loc[merged_permits['Permit APN'] == '24963032'].iloc[0]['ADDRESS']

'450 N 21ST ST  , SAN JOSE CA 95112-1747'

In [194]:
sj_assessor_parcels.loc[sj_assessor_parcels['ASSESSOR APN'] == '24963032'].iloc[0]['ADDRESS']

IndexError: single positional indexer is out-of-bounds

In [186]:
sj_assessor_parcels_av2.loc[sj_assessor_parcels_av2['ASSESSOR APN2'] == '43955021'].iloc[0]['ADDRESS']

'1930  HICKS AV  , SAN JOSE CA 95125'

In [195]:
assessor_parcels.loc[assessor_parcels['APN'] == '24963032'].iloc[0].to_dict()

IndexError: single positional indexer is out-of-bounds